In [1]:
%load_ext autoreload
%autoreload 2

## Table: Dataset stats

In [2]:
import sys
sys.path.append('..')

from src import data, models, functional
import torch
import os
import pandas as pd
import json

In [3]:
############################################################
model_name = "llama-13b"
device = "cuda" if torch.cuda.is_available() else "cpu"
results_dir = f"../results/known_samples/"
############################################################
os.makedirs(results_dir, exist_ok=True)

In [4]:

mt = models.load_model(model_name, fp16=True, device=device)

Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [5]:
dataset = data.load_dataset()
filtered = functional.filter_dataset_samples(
    mt=mt,
    dataset=dataset,
    n_icl_lm=functional.DEFAULT_N_ICL_LM,
    n_trials=3,
    batch_size=1,
)

filter dataset:   0%|          | 0/47 [00:00<?, ?it/s]

In [11]:
relations_by_name = {r.name: r for r in dataset.relations}
filtered_by_name = {r.name: r for r in filtered.relations}

samples_known = {}
for name in relations_by_name:
    relation_samples = set(relations_by_name[name].samples)
    filtered_samples = set(filtered_by_name[name].samples) if name in filtered_by_name else set()
    samples_known[name] = {
        "known": len(filtered_samples),
        "total": len(relation_samples),
        "known_samples": [
            {
                "subject": sample.subject,
                "object": sample.object,
            }
            for sample in filtered_samples
        ],
    }

with open(f"{results_dir}/{model_name}.json", "w") as f:
    json.dump(samples_known, f, indent=4)

In [12]:
df_json = [
    {
        "relation": key,
        "total": value["total"],
        model_name: value["known"],
    } for key, value in samples_known.items()]

df = pd.DataFrame(df_json)
df.to_excel(f"{results_dir}/{model_name}.xlsx", index=False)

In [8]:
dataset[0].properties.__dict__

{'relation_type': 'bias',
 'domain_name': 'characteristic',
 'range_name': 'gender',
 'symmetric': False,
 'fn_type': 'MANY_TO_ONE',
 'disambiguating': False}

In [17]:
from typing import Literal
property_key: Literal["relation_type", "fn_type", "disambiguating", "symmetric"] = "disambiguating"

category_wise = {}
for name in relations_by_name:
    property_value = relations_by_name[name].properties.__dict__[property_key]
    relation_samples = set(relations_by_name[name].samples)
    filtered_samples = set(filtered_by_name[name].samples) if name in filtered_by_name else set()
    if property_value not in category_wise:
        category_wise[property_value] = {
            "known": 0,
            "total": 0,
        }
    category_wise[property_value]["known"] += len(filtered_samples)
    category_wise[property_value]["total"] += len(relation_samples)

In [18]:
df_json = [
    {
        property_key: key,
        "total": value["total"],
        model_name: value["known"],
    } for key, value in category_wise.items()
]

pd.DataFrame(df_json)

,disambiguating,total,llama-13b
0,False,7515,5231
1,True,970,789
